In [6]:
import numpy as np
import pandas as pd
import queue
import matplotlib.pyplot as plt

from BayesianOptimization.bayesian_optimization import BayesianOptimization

from Backtest.backtest import Backtest
from Backtest.data import OHLCDataHandler
from MACD_ADXStrategy import MACD_ADXStrategy
from Backtest.open_json_gz_files import open_json_gz_files
from Backtest.generate_bars import generate_bars

In [7]:
def run_backtest(config, trading_data, ohlc_data, short_window, delta_window, window):
    short_window = int(short_window)
    long_window = short_window + int(delta_window)
    window = int(window)
    config['title'] = "MACD_ADXStrategy" + "_" + str(short_window) + "_" + str(long_window) + "_" + str(window)
    print("---------------------------------")
    print(config['title'])
    print("---------------------------------")

    events_queue = queue.Queue()

    data_handler = OHLCDataHandler(
        config, events_queue,
        trading_data = trading_data, ohlc_data = ohlc_data
    )
    strategy = MACD_ADXStrategy(config, events_queue, data_handler,
                            short_window=short_window, long_window=long_window,
                            window = window)
    backtest = Backtest(config, events_queue, strategy,
                        data_handler= data_handler)

    results = backtest.start_trading()
    
    return (results['cum_returns'][-1] - 1)

## Setting

In [8]:
config = {
    "csv_dir": "C:/backtest/Binance",
    "out_dir": "C:/backtest/results/MACD_ADXStrategy",
    "title": "MACD_ADXStrategy",
    "is_plot": False,
    "save_plot": False,
    "save_tradelog": False,
    "start_date": pd.Timestamp("2017-07-01T00:0:00", freq = "60" + "T"),    # str(freq) + "T"
    "end_date": pd.Timestamp("2018-04-01T00:00:00", freq = "60" + "T"),
    "equity": 1.0,
    "freq": 60,      # min
    "commission_ratio": 0.001,
    "suggested_quantity": None,     # None or a value
    "max_quantity": None,           # None or a value, Maximum purchase quantity
    "min_quantity": None,           # None or a value, Minimum purchase quantity
    "min_handheld_cash": None,      # None or a value, Minimum handheld funds
    "exchange": "Binance",
    "tickers": ['BTCUSDT']
}

In [9]:
# trading_data = {}
# for ticker in config['tickers']:
#     # trading_data[ticker] = open_gz_files(config['csv_dir'], ticker)
#     trading_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker + '.h5', key=ticker)

ohlc_data = {}
for ticker in config['tickers']:
    # ohlc_data[ticker] = generate_bars(trading_data, ticker, config['freq'])
    ohlc_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker +'_OHLC_60min.h5', key=ticker)

trading_data = None

In [10]:
gp_params = {"alpha": 1e-5}

## Acquisition Function "Upper Confidence Bound"

* Prefer exploitation (kappa=1.0)

* Prefer exploration (kappa=10)


## Acquisition Function "Expected Improvement"

* Prefer exploitation (xi=1e-4)
* Prefer exploration (xi=0.1)

In [11]:
BO = BayesianOptimization(
    run_backtest,
    {'short_window': (1, 120),
    'delta_window': (1, 120),
    'window': (1, 240)},
    is_int = [1, 1, 1], 
    invariant = {
        'config': config,
        'trading_data': trading_data,
        'ohlc_data': ohlc_data
    },
    random_state = 1
)
# BO.explore({
#     'short_window': np.arange(1, 120, 20),
#     'delta_window': np.arange(1, 120, 20)
#     },
#     eager=True)
BO.maximize(init_points=40, n_iter=60, acq="ei", xi=0.01, **gp_params)

Initialization
---------------------------------------------------------------------------
 Step |   Time |      Value |   delta_window |   short_window |    window | 
---------------------------------
MACD_ADXStrategy_38_100_216
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.6113710599
Sortino Ratio: 0.5464290786
Max Drawdown: 0.5441542864
Max Drawdown Duration: 273500
Total Returns: 0.0809700832
Annualized Returns: 0.1905690297
Compound Annual Growth Rate: 0.2011104996
---------------------------------
Trades: 32
Trade Winning: 40.62%
Average Trade: 0.85%
Average Win: 7.62%
Average Loss: -3.79%
Best Trade: 37.23%
Worst Trade: -12.22%
Worst Trade Date: 1 days 10:00:00
Avg Days in Trade: 1 days 10:16:52
-----------

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 2.8179935084
Sortino Ratio: 4.3386566985
Max Drawdown: 0.3011228387
Max Drawdown Duration: 273500
Total Returns: 1.0820582510
Annualized Returns: 2.5467034602
Compound Annual Growth Rate: 4.6181889867
---------------------------------
Trades: 48
Trade Winning: 50.00%
Average Trade: 1.94%
Average Win: 5.91%
Average Loss: -2.03%
Best Trade: 31.32%
Worst Trade: -7.51%
Worst Trade Date: 0 days 05:00:00
Avg Days in Trade: 0 days 23:18:45
---------------------------------
   10 | 00m04s |    1.08206 | 14.000000 | 17.000000 | 127.000000 | 
---------------------------------
MACD_ADXStrategy_2_118_24
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.2692404069
Sortino Ratio: 0.2863658541
Max Drawdown: 0.5474047770
Max Drawdown Duration: 273500
Total Returns: -0.0427032357
Annualized Returns: -0.1005051974
Compound Annual Growth Rate: -0.0976152200
---------------------------------
Trades: 39
Trade Winning: 46.15%
Average Trade: 0.30%
Average Win: 5.26%
Average Loss: -3.94%
Best Trade: 25.44%
Worst Trade: -10.41%
Worst Trade Date: 0 days 03:00:00
Avg Days in Trade: 1 days 09:41:32
---------------------------------
   19 | 00m08s |   -0.04270 | 22.000000 | 102.000000 | 137.000000 | 
---------------------------------
MACD_ADXStrategy_19_69_33
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
----------------------

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.9392797420
Sortino Ratio: 1.0116942759
Max Drawdown: 0.5515338602
Max Drawdown Duration: 273500
Total Returns: 0.2009188606
Annualized Returns: 0.4728772754
Compound Annual Growth Rate: 0.5386549700
---------------------------------
Trades: 51
Trade Winning: 43.14%
Average Trade: 0.74%
Average Win: 5.38%
Average Loss: -2.78%
Best Trade: 20.41%
Worst Trade: -9.96%
Worst Trade Date: 1 days 04:00:00
Avg Days in Trade: 1 days 03:07:04
---------------------------------
   28 | 00m07s |    0.20092 | 27.000000 | 69.000000 | 93.000000 | 
---------------------------------
MACD_ADXStrategy_88_141_75
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.4714412690
Sortino Ratio: 1.6012046021
Max Drawdown: 0.3782005665
Max Drawdown Duration: 273500
Total Returns: 0.4161305263
Annualized Returns: 0.9793937159
Compound Annual Growth Rate: 1.2679457579
---------------------------------
Trades: 37
Trade Winning: 48.65%
Average Trade: 1.42%
Average Win: 6.23%
Average Loss: -3.13%
Best Trade: 32.34%
Worst Trade: -16.16%
Worst Trade Date: 1 days 03:00:00
Avg Days in Trade: 1 days 09:30:49
---------------------------------
   37 | 00m09s |    0.41613 | 69.000000 | 106.000000 | 114.000000 | 
---------------------------------
MACD_ADXStrategy_10_36_78
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
-------------------------

...

---------------------------------
MACD_ADXStrategy_30_31_216
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.4823932765
Sortino Ratio: 1.9646731228
Max Drawdown: 0.3228413969
Max Drawdown Duration: 273500
Total Returns: 0.3516312290
Annualized Returns: 0.8275898887
Compound Annual Growth Rate: 1.0322840696
---------------------------------
Trades: 77
Trade Winning: 50.65%
Average Trade: 0.68%
Average Win: 3.17%
Average Loss: -1.87%
Best Trade: 25.72%
Worst Trade: -9.10%
Worst Trade Date: 0 days 04:00:00
Avg Days in Trade: 0 days 11:56:06
---------------------------------
   64 | 00m18s |    0.35163 | 1.000000 | 30.000000 | 216.000000 | 
---------------------------------
MACD_ADXStrategy_16_72_80
--------------------

---------------------------------
MACD_ADXStrategy_64_134_43
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.2402994657
Sortino Ratio: 1.4218354170
Max Drawdown: 0.3924301987
Max Drawdown Duration: 273500
Total Returns: 0.3150652115
Annualized Returns: 0.7415290845
Compound Annual Growth Rate: 0.9052464871
---------------------------------
Trades: 49
Trade Winning: 44.90%
Average Trade: 0.96%
Average Win: 5.38%
Average Loss: -2.63%
Best Trade: 29.40%
Worst Trade: -12.84%
Worst Trade Date: 1 days 01:00:00
Avg Days in Trade: 1 days 00:39:11
---------------------------------
   73 | 00m19s |    0.31507 | 70.000000 | 64.000000 | 43.000000 | 
---------------------------------
MACD_ADXStrategy_63_183_177
-----------------

---------------------------------
MACD_ADXStrategy_102_152_118
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.5659787383
Sortino Ratio: 0.6202520512
Max Drawdown: 0.5862628179
Max Drawdown Duration: 273500
Total Returns: 0.0593321333
Annualized Returns: 0.1396425277
Compound Annual Growth Rate: 0.1452887431
---------------------------------
Trades: 47
Trade Winning: 42.55%
Average Trade: 0.60%
Average Win: 6.01%
Average Loss: -3.41%
Best Trade: 31.92%
Worst Trade: -16.16%
Worst Trade Date: 1 days 03:00:00
Avg Days in Trade: 1 days 04:34:28
---------------------------------
   82 | 00m22s |    0.05933 | 50.000000 | 102.000000 | 118.000000 | 
---------------------------------
MACD_ADXStrategy_101_178_33
-------------

---------------------------------
MACD_ADXStrategy_120_143_79
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.0487079113
Sortino Ratio: 1.1718755919
Max Drawdown: 0.4493997249
Max Drawdown Duration: 273500
Total Returns: 0.2399937838
Annualized Returns: 0.5648429731
Compound Annual Growth Rate: 0.6590889247
---------------------------------
Trades: 52
Trade Winning: 48.08%
Average Trade: 0.76%
Average Win: 4.35%
Average Loss: -2.56%
Best Trade: 22.78%
Worst Trade: -10.99%
Worst Trade Date: 0 days 21:00:00
Avg Days in Trade: 0 days 21:45:00
---------------------------------
   91 | 00m20s |    0.23999 | 23.000000 | 120.000000 | 79.000000 | 
---------------------------------
MACD_ADXStrategy_76_147_31
----------------

   99 | 00m18s |   -0.12962 | 20.000000 | 109.000000 | 57.000000 | 
---------------------------------
MACD_ADXStrategy_13_45_176
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 2.6265423099
Sortino Ratio: 3.7599188013
Max Drawdown: 0.1980246283
Max Drawdown Duration: 171000
Total Returns: 0.7269326589
Annualized Returns: 1.7108893316
Compound Annual Growth Rate: 2.6178042446
---------------------------------
Trades: 40
Trade Winning: 50.00%
Average Trade: 1.73%
Average Win: 5.07%
Average Loss: -1.62%
Best Trade: 23.64%
Worst Trade: -5.73%
Worst Trade Date: 0 days 17:00:00
Avg Days in Trade: 1 days 01:31:30
---------------------------------
  100 | 00m14s |    0.72693 | 32.000000 | 13.000000 | 176.000000 | 


In [12]:
print(BO.res['max'])

{'max_val': 2.0517390261286943, 'max_params': {'short_window': 22.0, 'delta_window': 18.0, 'window': 78.0}}


In [13]:
BO.maximize(init_points=0, n_iter=60, acq="ei", xi=0.0001, **gp_params)

Bayesian Optimization
---------------------------------------------------------------------------
 Step |   Time |      Value |   delta_window |   short_window |    window | 
---------------------------------
MACD_ADXStrategy_17_36_120
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 2.6479514021
Sortino Ratio: 3.9973215600
Max Drawdown: 0.2334441839
Max Drawdown Duration: 172700
Total Returns: 0.9515617017
Annualized Returns: 2.2395702597
Compound Annual Growth Rate: 3.8243189291
---------------------------------
Trades: 52
Trade Winning: 51.92%
Average Trade: 1.67%
Average Win: 5.08%
Average Loss: -2.02%
Best Trade: 31.29%
Worst Trade: -7.04%
Worst Trade Date: 0 days 19:00:00
Avg Days in Trade: 0 days 22:36:55
------

---------------------------------
MACD_ADXStrategy_4_72_142
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.8178048936
Sortino Ratio: 1.0748944682
Max Drawdown: 0.3631128685
Max Drawdown Duration: 273500
Total Returns: 0.1579330205
Annualized Returns: 0.3717069479
Compound Annual Growth Rate: 0.4121591496
---------------------------------
Trades: 46
Trade Winning: 26.09%
Average Trade: 0.76%
Average Win: 8.91%
Average Loss: -2.12%
Best Trade: 26.23%
Worst Trade: -10.06%
Worst Trade Date: 0 days 01:00:00
Avg Days in Trade: 1 days 02:57:23
---------------------------------
  110 | 00m13s |    0.15793 | 68.000000 | 4.000000 | 142.000000 | 
---------------------------------
MACD_ADXStrategy_88_145_96
-------------------

---------------------------------
MACD_ADXStrategy_99_133_199
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.3407215622
Sortino Ratio: 0.3070572050
Max Drawdown: 0.5211631059
Max Drawdown Duration: 273500
Total Returns: -0.0116205743
Annualized Returns: -0.0273498739
Compound Annual Growth Rate: -0.0271350718
---------------------------------
Trades: 35
Trade Winning: 57.14%
Average Trade: 0.33%
Average Win: 3.42%
Average Loss: -3.80%
Best Trade: 14.77%
Worst Trade: -18.98%
Worst Trade Date: 2 days 21:00:00
Avg Days in Trade: 1 days 11:05:09
---------------------------------
  119 | 00m15s |   -0.01162 | 34.000000 | 99.000000 | 199.000000 | 
---------------------------------
MACD_ADXStrategy_20_40_77
--------------

---------------------------------
MACD_ADXStrategy_91_207_202
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.6314824787
Sortino Ratio: 0.7684028943
Max Drawdown: 0.4573610531
Max Drawdown Duration: 273500
Total Returns: 0.0920029761
Annualized Returns: 0.2165357524
Compound Annual Growth Rate: 0.2301627399
---------------------------------
Trades: 27
Trade Winning: 33.33%
Average Trade: 0.75%
Average Win: 7.93%
Average Loss: -2.83%
Best Trade: 20.95%
Worst Trade: -12.32%
Worst Trade Date: 2 days 15:00:00
Avg Days in Trade: 1 days 23:02:13
---------------------------------
  128 | 00m18s |    0.09200 | 116.000000 | 91.000000 | 202.000000 | 
---------------------------------
MACD_ADXStrategy_103_179_70
--------------

---------------------------------
MACD_ADXStrategy_2_111_175
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.2202492977
Sortino Ratio: 1.6296546235
Max Drawdown: 0.3574017173
Max Drawdown Duration: 273500
Total Returns: 0.2871365881
Annualized Returns: 0.6757970208
Compound Annual Growth Rate: 0.8113802195
---------------------------------
Trades: 42
Trade Winning: 33.33%
Average Trade: 1.00%
Average Win: 7.21%
Average Loss: -2.11%
Best Trade: 26.14%
Worst Trade: -6.94%
Worst Trade Date: 0 days 15:00:00
Avg Days in Trade: 1 days 04:47:09
---------------------------------
  137 | 00m16s |    0.28714 | 109.000000 | 2.000000 | 175.000000 | 
---------------------------------
MACD_ADXStrategy_65_72_4
--------------------

---------------------------------
MACD_ADXStrategy_83_153_193
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.6963057910
Sortino Ratio: 0.6594403015
Max Drawdown: 0.4652891909
Max Drawdown Duration: 273500
Total Returns: 0.1096415773
Annualized Returns: 0.2580494942
Compound Annual Growth Rate: 0.2774409914
---------------------------------
Trades: 24
Trade Winning: 37.50%
Average Trade: 0.90%
Average Win: 8.62%
Average Loss: -3.74%
Best Trade: 18.55%
Worst Trade: -10.14%
Worst Trade Date: 2 days 04:00:00
Avg Days in Trade: 2 days 03:47:30
---------------------------------
  146 | 00m17s |    0.10964 | 70.000000 | 83.000000 | 193.000000 | 
---------------------------------
MACD_ADXStrategy_15_39_79
-----------------

---------------------------------
MACD_ADXStrategy_28_105_223
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.1418719811
Sortino Ratio: 0.1466029005
Max Drawdown: 0.5524763411
Max Drawdown Duration: 273500
Total Returns: -0.0611752970
Annualized Returns: -0.1439805486
Compound Annual Growth Rate: -0.1380627895
---------------------------------
Trades: 39
Trade Winning: 30.77%
Average Trade: 0.34%
Average Win: 7.39%
Average Loss: -2.79%
Best Trade: 36.75%
Worst Trade: -14.91%
Worst Trade Date: 1 days 05:00:00
Avg Days in Trade: 1 days 04:03:05
---------------------------------
  155 | 00m17s |   -0.06118 | 77.000000 | 28.000000 | 223.000000 | 
---------------------------------
MACD_ADXStrategy_28_97_148
-------------

In [14]:
print(BO.res['max'])

{'max_val': 2.0517390261286943, 'max_params': {'short_window': 22.0, 'delta_window': 18.0, 'window': 78.0}}


In [15]:
Target = pd.DataFrame({'Parameters': BO.X.tolist(), 'Target': BO.Y})
# Target.to_csv(config['out_dir'] + "/target_ei.csv")
Target.sort_values(by = "Target")

,Parameters,Target
40,"[100.0, 38.0, 1.0]",-0.778741
38,"[8.0, 112.0, 4.0]",-0.572465
50,"[1.0, 4.0, 139.0]",-0.509279
137,"[65.0, 7.0, 4.0]",-0.444309
117,"[109.0, 10.0, 8.0]",-0.418017
49,"[113.0, 92.0, 6.0]",-0.410692
97,"[46.0, 40.0, 7.0]",-0.363098
42,"[120.0, 120.0, 224.0]",-0.255784
32,"[95.0, 42.0, 217.0]",-0.204366
98,"[109.0, 20.0, 57.0]",-0.129619


## Acquisition Function "Probability of Improvement"

* Prefer exploitation (xi=1e-4)
* Prefer exploration (xi=0.1)